## Overall workflow
1. action链接数量找到所有核心概念
2. 复合概念，构造概念依赖树
3. 输入历史材料，涉及概念过滤，建立与核心概念的依赖关系
4. 概念聚类（文章中可能包含多个支线，分离开）
5. 生成句子（限制与规则）
    1）从已知概念出发
    2）根据预期长度，对不同聚类的概念进行分配权重，覆盖所有cluster

In [147]:
import nltk
import re
import tqdm
from pathlib import Path
from collections import Counter, namedtuple
stop_words = set(nltk.corpus.stopwords.words('english'))
from lexer import Sentence
import termcolor    # 命令行变色
import inflect      # 判断单词复数

In [49]:
Default_IDF_Value

7.513607609220388

$1+1$


# 11.4
文本摘要和课程大纲的区别
规则/约束
验证，课程大纲是需要规则的
规则的作用，效果（评价）
为什么需要规则？

否则，学生以现有知识水平理解不了。
满足了一些额外的评价指标（课程大纲贴切的），合理性

In [268]:


# TODO: 数据预处理
# '-'删掉，但是first-normal-form，先变成空格，再判断每个词是不都是词，如果存在非单词的，就合并
# nltk能不能判断复数，给合并一下

import inflect

p = inflect.engine()

word1 = "cat"
word2 = "cats"

# 判断是否为复数
if p.singular_noun(word2) == word1:
    print(f"{word2} 是 {word1} 的复数形式")
else:
    print(f"{word2} 不是 {word1} 的复数形式")

# TODO: 词干相同是否也聚类？

cats 是 cat 的复数形式


In [275]:
def is_same_wo_singular(word1, word2):
    

A, is, attr, B



A of B, is C

(A of B)
(B is C)

(A of B) is C

captain of China is C

A   of  cons    C   1

sentence 1
sentence 2



?p.singular_noun

Signature:
p.singular_noun(
    text: 'Word',
    count: 'Optional[Union[int, str, Any]]' = None,
    gender: 'Optional[str]' = None,
) -> 'Union[str, Literal[False]]'
Docstring:
Return the singular of text, where text is a plural noun.

If count supplied, then return the singular if count is one of:
    1, a, an, one, each, every, this, that or if count is None

otherwise return text unchanged.

Whitespace at the start and end is preserved.

>>> p = engine()
>>> p.singular_noun('horses')
'horse'
>>> p.singular_noun('knights')
'knight'

Returns False when a singular noun is passed.

>>> p.singular_noun('horse')
False
>>> p.singular_noun('knight')
False
>>> p.singular_noun('soldier')
False
File:      ~/anaconda3/envs/sln/lib/python3.11/site-packages/inflect/__init__.py
Type:      method

: 

In [267]:
!pip install inflect

509237.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


In [48]:
# TODO: 移除多少词？
idf_value_mapper = {}
Default_IDF_Value = 0
for line in Path('idf.txt').read_text().split('\n'):
    if not line: continue
    word, value = line.split('\t')
    value = float(value)
    idf_value_mapper[word] = value

    Default_IDF_Value += value

Default_IDF_Value /= len(idf_value_mapper)

def get_idf_value(string: str):
    idf_values = []
    
    for word in (words := string.split(' ')):
        if word in idf_value_mapper:
            idf_values.append(idf_value_mapper[word])

    if len(idf_values):
        return sum(idf_values) / len(idf_values)
    return Default_IDF_Value

In [59]:
ActionType = 'action'
AttributeType = 'attribution'
ConjunctiveType = 'conjunction'
ConstraintType = 'constraint'
SequentialType = 'sequential'

Link = namedtuple('Link', ['pre', 'ind', 'rtype', 'post', 'position'])

def _preprocess_node(node):
        '''
        Filter the meaningful words.
        '''
        node = node.replace('-', '')
        node = node.lower()
        words = []
        word_pattern = re.compile('(\d*[a-zA-Z]{3,}\d*|<b>)')
        
        for word, tag in nltk.pos_tag(nltk.tokenize.word_tokenize(node)):
            if tag not in ['NN', 'NNS', 'VBG', 'JJ', 'CD', 'VBZ', 'VBN', 'RB', 'JJS', 'JJR', 'VB', 'RBR', 'RBS', 'VBD', 'PDT', 'VBP', 'RP', 'IN']:
                continue
            # the word starts with a alphabet
            if not word_pattern.match(word) or word in stop_words or len(word) <= 2:
                continue
            words.append(word)

        return ' '.join(words)

def load_links(file):
    links = []
    for line in tqdm.tqdm(file.read_text().split('\n')):
        if not line: continue
        pre, ind, rtype, post, position = line.split('\t')
        
        pre, ind, post = _preprocess_node(pre), _preprocess_node(ind), _preprocess_node(post)
        if not all([pre, post]) or pre == post: continue
        
        links.append(Link(pre, ind, rtype, post, position))
    return links

core_concepts = set()
action_link_counter = {}
tf_link_counter = {}
main_links = load_links(Path('rsm.triplets'))
for pre, ind, rtype, post, _ in main_links:
    if rtype == ActionType:
        action_link_counter[pre] = action_link_counter.get(pre, []) + [ind]
    tf_link_counter[pre] = tf_link_counter.get(pre, []) + [ind]

100%|██████████| 24492/24492 [00:10<00:00, 2272.86it/s]


In [28]:
for word, inds in sorted(action_link_counter.items(), key = lambda x: -len(x[1])):
    print(word, len(inds))

resource space model 86
resource space 74
resources 43
semantic link network 43
resource 39
node 34
peer 32
operations 25
rs2 24
users 23
axis 21
people 20
one 19
chapter 17
approach 14
network 14
theorem 14
coordinate 13
semantic link 13
tree 13
resource spaces 12
space 12
example 12
query 12
probabilistic resource space model 12
nodes 12
peers 12
point 11
rsm 11
resource indices 11
external relationships 10
resource space calculus 10
best algorithms 10
hierarchy semantics 10
structured rsm 10
relational database model 9
neighbors 9
one peer 9
applications 8
user 8
operation 8
rs1 8
alpha expression 8
space dimension 8
new node 8
object 7
system 7
coordinates 7
data cube 7
algorithm 7
networks 7
community 7
files 6
classification 6
method 6
concepts 6
way 6
rs2rs2 6
axes 6
two resource spaces 6
relational data model 6
process 6
relational calculus 6
results 6
worst case 6
resource index 6
messages 6
goods 5
world wide web 5
common axes 5
view 5
dimension 5
designers 5
redundancy 5
cat

In [61]:
for word, inds in sorted(tf_link_counter.items(), key = lambda x: -len(x[1])):
    print(word, len(inds))

resource space model 228
resource space 224
resources 167
coordinates 109
axis 105
resource 100
semantic link network 95
peer 95
example 93
point 88
operations 86
number 83
coordinate 78
node 76
one 71
set 68
users 51
sln 48
nodes 48
query 47
resource spaces 43
type 40
axes 39
union 38
peers 38
definition 37
rsm 37
probability 37
classification 36
tree 35
points 34
theorem 33
people 32
neighbors 32
approach 31
table 31
concepts 30
difference 30
class 29
space 28
structured rsm 28
complexity 27
distribution 26
information 25
semantic link 25
network 25
external relationships 25
bit string 25
hierarchy semantics 24
level 23
semantics 23
probabilistic resource space model 23
applications 22
view 22
disjoin 22
results 22
resource indices 22
algorithm 21
category 21
process 21
semantic links 21
relationship 21
distance 21
searching complexity 21
categories 20
data 20
relational database model 20
common axes 20
selection 20
alpha expression 20
way 19
object 19
community 19
merge 19
dimension

Top 100 / 1127 Core concepts, the weight is $|Action(n)| * IDF(n)$

In [63]:

print(len(action_link_counter))
core_concepts = set()
concept_weight_mapper = {}

for index, (word, inds) in enumerate(sorted(action_link_counter.items(), key = lambda x: -len(x[1]) * get_idf_value(x[0]))):
    print(word, len(inds))
    if index <= 100:
        core_concepts.add(word)
        concept_weight_mapper[word] = len(action_link_counter[word]) * get_idf_value(word)

1127
peer 32
resource space model 85
resource space 71
resource 39
semantic link network 43
node 34
rsm 11
resources 41
axis 21
chapter 17
peers 12
operations 24
theorem 14
coordinate 13
users 23
structured rsm 10
people 20
sln 5
coordinates 9
plications 5
gossip 5
neighbors 9
resource indices 11
query 12
nodes 12
resource space calculus 10
axes 6
alpha expression 8
one peer 9
timer 4
network 14
tree 13
semantic link 13
resource spaces 10
hierarchy semantics 10
goods 5
disjoin 4
external relationships 10
designers 5
probabilistic resource space model 12
mbr 4
relational calculus 6
ddl 3
introducer 3
data cube 7
relational database model 9
space 12
operation 8
virus 4
messages 6
designer 4
zone 4
rol 3
gossip mechanisms 4
data warehouse 5
redundancy 5
one 18
point 11
files 6
query initiator 4
rsm schema tree 4
best algorithms 10
customers 4
space dimension 8
approach 14
community 7
routing process 5
corre 3
storing 4
user 8
rsn 2
object 7
networks 7
new peer 5
common axes 5
quotient sem

In [62]:
print(len(action_link_counter))
for word, inds in sorted(tf_link_counter.items(), key = lambda x: -len(x[1]) * get_idf_value(x[0])):
    print(word, len(inds))

1127
coordinates 109
peer 95
resource space 224
resource space model 228
axis 105
sln 48
resources 167
coordinate 78
rsm 37
operations 86
resource 100
peers 38
semantic link network 95
node 76
axes 39
point 88
disjoin 22
structured rsm 28
theorem 33
query 47
nodes 48
resource spaces 43
neighbors 32
union 38
users 51
mbr 18
example 93
number 83
ops 13
zone 18
definition 37
cij 11
chapter 18
concepts 30
one 71
set 68
tree 35
bit string 25
rsn 9
plications 10
alpha expression 20
goods 13
people 32
points 34
probability 37
completeness 17
external relationships 25
hierarchy semantics 24
relational calculus 16
resource indices 22
join 15
merge 19
classification 36
complexity 27
common axes 20
expressiveness 13
type 40
intersection 17
lgn 9
designers 12
zones 11
searching complexity 21
calculus 12
introducer 7
class 29
apartment 9
projection 16
semantics 23
extended cartesian product 16
semantic link 25
rol 8
resource space calculus 18
difference 30
bit 15
fanout 7
bit strings 16
space 28
re

In [ ]:
# compound concepts: find the abstract relations

for pre, ind, rtype, post in triplets:
    if rtype != ConstraintType:
        continue

    if pre in core_concepts:
        if get_idf_value(post) > :
            pass
    
    if post in core_concepts:
        pass

## dependency chain

1. 一个语义链内，前面的词依赖于后面的词
2. 一句话内，前面的词依赖于后面的词，依赖强度取决于间隔了多少个语义节点

- [ ] 是否考虑constraint link
- [x] 使用idf阈值硬过滤效果不好，把idf软融入到dependency chain中

$$
Dependency(Pre, Post) = \frac{}{}
$$

In [67]:
def get_dependency_matrix(link_list):
    dependency_matrix = {}
    position_mapper = {}

    for link in link_list:
        position = link.position
        position_mapper[position] = position_mapper.get(position, []) + [link]

    for section, links in position_mapper.items():
        pre_list = []
        # TODO: relation type给一个系数
        for pre, ind, rtype, post, position in links:
            # 离的越近，（_pre, post）后面的加权越多，所以pre_list需要逆转
            for index, _pre in enumerate(pre_list, 1):
                dependency_matrix[(_pre, post)] = dependency_matrix.get((_pre, post), 0) + (1 / index) * (get_idf_value(_pre) + get_idf_value(post)) / 2

            # TODO: 更完善的计算
            dependency_matrix[(pre, post)] = dependency_matrix.get((pre, post), 0) + (get_idf_value(pre) + get_idf_value(post) + get_idf_value(ind)) / 3
            pre_list.append(pre)

    return dependency_matrix

main_dependency_matrix = get_dependency_matrix(main_links)

# TODO: 删除反向依赖值
core_dependency_matrix = {}
for pair, value in main_dependency_matrix.items():
    if pair[0] not in core_concepts or pair[1] not in core_concepts or pair[0] == pair[1]:
        continue
    core_dependency_matrix[pair] = value

for pair, value in sorted(core_dependency_matrix.items(), key=lambda x: -x[1])[:10]:
    print(pair, value)

('coordinates', 'axis') 174.5541710193902
('resource space model', 'semantic link network') 124.92877236532084
('node', 'sln') 114.02327536406798
('resources', 'resource space') 102.04500295363385
('alpha expression', 'axes') 92.66888188510747
('rsm', 'sln') 90.19767934870063
('peer', 'peers') 82.33344673948027
('plications', 'resources') 80.95392934870064
('resource space', 'axis') 78.22188188510746
('coordinate', 'axis') 76.17609710354823


In [216]:
# Construct Dependency Chain

# dependency chain
class DependencyNode:

    def __init__(self, content: str):
        self.content = content
        self.next = []
        self.pre = []
    
    def __hash__(self):
        return hash(self.content)
    
    def __eq__(self, query):
        return self.content == query.content

    def has_next(self, query):
        # query is a chainNode
        searched = set()
        pending = {self}
        
        while pending:
            node = pending.pop()
            searched.add(node)
            
            if node == query:
                return True
            for next in node.next:
                if next not in searched:
                    pending.add(next)
        
        return False

    def __str__(self):
        return self.content
        # string = f'{self.content}'
        # strings = []
        # for node in self.next:
        #     strings.append(
        #         f'{string} ----> {str(node)}'
        #     )
        # if not strings:
        #     return string
        # return '\n'.join(strings)
        
    def __repr__(self):
        return f'ChainNode({self.content})'


class DependencyForest:

    def __init__(self):
        self.roots = []
        self.string_to_node = {}

    def _append(self, pre_node, post_node):
        # 如果存在post_node -> ... -> pre_node的依赖链路，则不添加
        if post_node.has_next(pre_node):
            return False
        
        return True

    def append(self, pair):
        # pair: (pre, post), where pre is depended on post
        pre, post = pair
        pre_node = self.search(pre)
        post_node = self.search(post)

        pre_as_root = False

        if pre_node is not None and post_node is not None:
            pass
        elif pre_node is not None and post_node is None:
            post_node = DependencyNode(post)
        elif pre_node is None and post_node is not None:
            pre_node = DependencyNode(pre)
            pre_as_root = True
        elif pre_node is None and post_node is None:
            pre_node = DependencyNode(pre)
            post_node = DependencyNode(post)
            pre_as_root = True

        # 如果存在post_node -> ... -> pre_node的依赖链路，则不添加
        if post_node.has_next(pre_node):
            return
        
        pre_node.next.append(post_node)
        post_node.pre.append(pre_node)

        # pre没有依赖，添加root节点
        if pre_as_root:
            self.roots.append(pre_node)

        # post如果是root，移除
        if post_node in self.roots:
            self.roots.remove(post_node)
        
        self.string_to_node[pre_node.content] = pre_node
        self.string_to_node[post_node.content] = post_node

    def search(self, query):
        return self.string_to_node.get(query, None)

    def node_search(self, node, query):
        # print('searching, ', node.content, query)
        if node.content == query:
            return node
        for _node in node.next:
            if (result := self.node_search(_node, query)):
                return result

    def get_node_sequence(self, from_concept, to_concept, forward=True) -> list[str]:
        '''
        forward: True, search process is from `from_node` to `to_node`
            False, which is from `to_node` backward to `from_node`
        
        to_concept: if it is none, which represents the endless iteration
        return the middle concepts from `from_node` to `to_node`
        return [] if the dependency relations does not exist
        
        TODO: 遍历依赖强度顺序添加
        notice that forward=True and forward=False may be inconsistent
        '''
        if (forward and not from_concept) or (not forward and not to_concept):
            raise Exception('get node sequence invalid')
        
        from_node = self.string_to_node[from_concept] if from_concept else None
        to_node = self.string_to_node[to_concept] if to_concept else None
        
        iterated_nodes = set()
        paths = []
        
        if forward:
            queue = [(from_node, [from_node])]
            # 找到所有dependency sequence
            while queue:
                node, path = queue.pop()
                iterated_nodes.add(node)
                
                for next in node.next:
                    if next not in iterated_nodes:
                        iterated_nodes.add(node)
                        new_path = path + [next]
                        if (not to_node and len(next.next) == 0) or (to_node and next == to_node):
                            paths.append(new_path)
                        else:
                            queue.append(
                                (next, new_path)
                            )
        
        # 从后向前
        if not forward:
            queue = [(to_node, [to_node])]
            # 找到所有dependency sequence
            while queue:
                node, path = queue.pop()
                iterated_nodes.add(node)
                
                for pre in node.pre:
                    if pre not in iterated_nodes:
                        iterated_nodes.add(node)
                        
                        new_path = [pre] + path
                        if (not from_node and len(pre.pre) == 0) or (from_node and pre == from_node):
                            paths.append(new_path)
                        else:
                            queue.append(
                                (pre, new_path)
                            )
        
        # 按照path的长度，从长到短进行排序，生成摘要的时候可以动态选择，根据句子预算（sentence budget）
        return sorted(paths, key=lambda p: len(p))

    def __str__(self):
        string = ''
        for root in self.roots:
            string += str(root) + '\n==================\n'
        return string

def construct_dependency_foreset(dependency_matrix):
    dependency_forest = DependencyForest()
    # 按照频繁集挖掘的策略来运行
    for index, (pair, value) in enumerate(
        tqdm.tqdm(sorted(dependency_matrix.items(), key=lambda x: -x[1]))
    ):
        # pair[0] 依赖 pair[1]
        dependency_forest.append(pair)
        
    return dependency_forest

In [217]:
core_dependency_forest = construct_dependency_foreset(core_dependency_matrix)
core_dependency_forest.roots[0].content
core_dependency_forest.get_node_sequence('object', 'community')
# print(core_dependency_forest.get_node_sequence('object', '', forward=True))
# core_dependency_forest.get_node_sequence('', 'community', forward=False)
# core_dependency_forest.get_node_sequence('community', '', forward=False)

100%|██████████| 693/693 [00:00<00:00, 94991.75it/s]


[[ChainNode(object),
  ChainNode(applications),
  ChainNode(peer),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(peers),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(one peer),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(peer),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(one peer),
  ChainNode(one),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(nodes),
  ChainNode(semantic link),
  ChainNode(semantic link network),
  ChainNode(peer),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(example),
  ChainNode(space),
  ChainNode(coordinate),
  ChainNode(introducer),
  ChainNode(community)],
 [ChainNode(object),
  Chain

In [218]:
core_dependency_forest = construct_dependency_foreset(core_dependency_matrix)
core_dependency_forest.get_node_sequence('object', 'community', forward=False)

100%|██████████| 693/693 [00:00<00:00, 94411.69it/s]


[[ChainNode(object),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(one peer),
  ChainNode(one),
  ChainNode(community)],
 [ChainNode(object),
  ChainNode(databases),
  ChainNode(applications),
  ChainNode(resources),
  ChainNode(one peer),
  ChainNode(one),
  ChainNode(community)]]

In [75]:
def breath_first_iteration(root):
    searched = set()
    pending = {root}
    mapper = {}
        
    while pending:
        parent = pending.pop()
        searched.add(parent)

        mapper[parent.content] = []

        for next in parent.next:
            if next not in searched and next not in pending:
                pending.add(next)
                mapper[parent.content].append(next.content)

    # print(mapper)
    print(f'{len(searched)} / {len(core_concepts)} concepts')
    # return len(searched), print_tree(root.content, mapper, '')
    return print_tree(root.content, mapper, '')
        
def print_tree(root, mapper, string, last=True, header='') -> str:
    elbow = "└──"
    pipe = "│  "
    tee = "├──"
    blank = "   "
    string += (header + (elbow if last else tee) + root) + '\n'
    children = mapper[root]
    for i, c in enumerate(children):
        string += print_tree(c, mapper, '', header=header + (blank if last else pipe), last=i == len(children) - 1)
    return string

# len(dependency_chain.chains[0])
print(breath_first_iteration(dependency_forest.roots[0]))

51 / 101 concepts
└──object
   ├──databases
   └──applications
      ├──networks
      ├──peer
      │  ├──peers
      │  ├──query
      │  ├──network
      │  ├──community
      │  │  └──tree
      │  └──issuing message
      ├──files
      ├──resource spaces
      ├──operation
      │  ├──ops
      │  ├──disjoin
      │  │  └──resource space
      │  │     ├──two resource spaces
      │  │     ├──axes
      │  │     ├──sln
      │  │     ├──semantic link network
      │  │     │  └──redundancy
      │  │     ├──cij
      │  │     ├──designer
      │  │     ├──semantic link
      │  │     ├──rsn
      │  │     └──space
      │  ├──operations
      │  ├──example
      │  │  ├──point
      │  │  ├──mbr
      │  │  └──profes sor
      │  ├──alpha expression
      │  └──resource space calculus
      │     ├──relational calculus
      │     ├──coordinate
      │     │  ├──introducer
      │     │  └──nodes
      │     └──axis
      │        └──space dimension
      ├──messages
      │  └──

Load the history materials and build the dependency forest

Merge the main dependency forest with the history dependency forest.

In [79]:
history_links = load_links(Path('foundations_of_database.triplets'))

  0%|          | 0/71910 [00:00<?, ?it/s]

100%|██████████| 71910/71910 [00:31<00:00, 2277.81it/s]


In [91]:
history_dependency_matrix = get_dependency_matrix(history_links)
# select the top xxx concepts
history_dependency_forest = construct_dependency_foreset(history_dependency_matrix)

100%|██████████| 87128/87128 [01:15<00:00, 1153.50it/s]


In [ ]:
history_dependency_forest.roots

In [141]:
def extract_common_forest(main_foreset, history_forest):
    '''
    merge the side_chain to main_chain
    '''
    pairs = []
    iterated_nodes = set()
    queue = [(root, []) for root in history_forest.roots]
    pair_sequences = []
    # 找到所有dependency sequence
    while queue:
        node, pairs = queue.pop()
        iterated_nodes.add(node)
        for next in node.next:
            if next not in iterated_nodes:
                queue.append(
                    (next, pairs + [(node, next)])
                )
            else:
                pair_sequences.append(pairs)
        if len(node.next) == 0:
            pair_sequences.append(pairs)

    filtered_pairs = []
    # 找到最短路径
    for sequence in tqdm.tqdm(pair_sequences):
        start = -1; end = -1
        for index, (pre, post) in enumerate(sequence):
            if pre.content in main_foreset.string_to_node and start == -1:
                start = index
            if post.content in main_foreset.string_to_node and start != -1:
                end = index + 1

        if end != -1:
            # 获取中间所有concepts
            filtered_pairs.append(sequence[start:end])

    print(len(filtered_pairs))
    
    for pre, post in filtered_pairs[0]:
        print(pre.content, '->', post.content)

    # for filtered_pair in filtered_pairs:
    #     main_dependency_matrix.append(filtered_pair)
    
    common_forest = DependencyForest()
    for pairs in tqdm.tqdm(filtered_pairs):
        for pre, post in pairs:
            common_forest.append((pre.content, post.content))
    return common_forest

common_history_forest = extract_common_forest(core_dependency_forest, history_dependency_forest)

100%|██████████| 88946/88946 [00:00<00:00, 718005.96it/s]


3443
databases -> resolution
resolution -> head
head -> oid
oid -> form
form -> semantic information
semantic information -> designer


100%|██████████| 3443/3443 [00:00<00:00, 7298.74it/s] 


In [113]:
'designer' in core_dependency_forest.string_to_node

True

4. Decompose / 解耦 dependency tree

* 把core concepts按条目分为多条学习路径
* 把core concepts的不同路径，直接忽视掉某些不重要的节点

$$
Importance(D) = \alpha * Importance(D) + \beta * \sum_{T_i \in Incoming}\frac{PR(T_i)}{|Incoming(D)|} + (1 - \beta) * \sum_{T_i \in Outgoing}\frac{PR(T_i)}{|Outgoing(D)|} $$
$$
\alpha = \frac{|Outgoing(D)| + |Incoming(D)|}{N}
$$

In [135]:
# 计算所有节点的重要度
importances = {}

# importance init
for concept in core_dependency_forest.string_to_node.keys():
    importances[concept] = 1 # or the count of action links

# importance transfer
N = len(core_dependency_forest.string_to_node.keys())
beta = 0.4

# the ending condition: 
for i in range(9):
    for concept in core_dependency_forest.string_to_node.keys():
        incoming = [n.content for n in core_dependency_forest.string_to_node[concept].pre]
        outgoing = [n.content for n in core_dependency_forest.string_to_node[concept].next]
    
        importances[concept] *= (len(incoming) +len(outgoing)) / N
    
        importances[concept] += beta * sum(importances[p] for p in incoming) / (len(incoming) + 1)
        importances[concept] += (1 - beta) * sum(importances[p] for p in incoming) / (len(incoming) + 1)

sorted(importances.items(), key=lambda x: -x[1])

[('resource space', 0.03009720930540835),
 ('axis', 0.024989215630144106),
 ('semantic link network', 0.020338168970456684),
 ('sln', 0.017983957288784193),
 ('rsn', 0.017254492358874565),
 ('peer', 0.01716773903677528),
 ('designer', 0.015230931568747915),
 ('query', 0.015225757572200911),
 ('relational calculus', 0.015100259730818903),
 ('axes', 0.014806191779589748),
 ('two resource spaces', 0.014596285155379637),
 ('network', 0.01443309836567343),
 ('resource spaces', 0.014203950314280185),
 ('nodes', 0.013823117549164116),
 ('semantic link', 0.013599009098811234),
 ('peers', 0.013232022714070948),
 ('gossip mechanisms', 0.013176542512581775),
 ('resources', 0.012819400278791804),
 ('coordinate', 0.012567199841158526),
 ('cij', 0.01092935661906289),
 ('networks', 0.009870856181365212),
 ('community', 0.009146195106133686),
 ('common axes', 0.008662267451541993),
 ('redundancy', 0.008404179767541846),
 ('tree', 0.007548555738884385),
 ('space', 0.007109793256949995),
 ('example', 0.

In [ ]:
# calculate the importances of pairs
# 入链可以重复，出链不能重复

pair_importances = []
for pre, post, dependency_coef in pairs:
    pair_importances.append(
        pair, (importances[pre] + importances[post]) * dependency_coef
    )

chains = []
solved_nodes = []

the_most_important_pair = sorted(pair_importances, key=lambda x: x[1])[0]

while True:
    chain = select_a_chain(the_most_important_pair, 0.3, all_pairs)
    

def select_a_chain(start_pair, threshold, all_pairs):
    chain = DependencyChain()
    query_pre, query_post = start_pair

    for pre, post in all_pairs:
        if post == query_pre and importance > threshold:
            chain.append((pre, post))

            # 所有包含pre节点的pair全部降权重
            importance *= 0.5

        if pre == query_post and importance > threshold:
            chain.append((pre, post))

            # 移除所有包含post节点的pair

# 生成课程大纲

### 推理句子选择：限制与规则

输入：预期的句子数量


4. 可读性

分为三段；将句子数量按照某种固定比例分配在三段中。如10%，20%，70%；也可以按照$\frac{|basis-concepts|}{|core-concepts|}$的比例计算
    1) 只包含basis concepts; 2) 包含basis concepts和core conceptes; 3) 只包含core concepts


1. 信息量

ranking 句子的方法

2. 连贯性

所有句子必须遵循依赖关系，不能跳concepts，结合【位置】信息。

3. 相关性

title, section title作为指导信息

5. 简洁性

针对连贯性抽取整个句子，根据语义链，保留核心内容。即不直接抽取整个句子，抽取出包含该依赖关系的重要部分作为半句话，与其他半句进行连接。

一页A4纸，多少句子/词：有应用背景的文本摘要

In [150]:
# load rsm sentences

position_section_mapper = {}

for path in sorted(Path('rsm_coref/').glob('*')):
    for para_index, paragraph in enumerate(tqdm.tqdm(path.read_text().split('\n'), desc=path.name), 1):
        for sent_index, sentence_text in enumerate(nltk.tokenize.sent_tokenize(paragraph), 1):
            # sentence = Sentence(sentence_text)
            words = nltk.word_tokenize(sentence_text)
            position = f"{path.name.split(' ')[0]}-{para_index}-{sent_index}"
            position_section_mapper[position] = (
                sentence_text, words
            )

1.2.7 Visualized Resource Locating: 100%|██████████| 3/3 [00:00<00:00, 4232.40it/s]
1.3 Application Scenarios of the Resource Space Model: 100%|██████████| 1/1 [00:00<00:00, 26214.40it/s]
1.3.4 Geographical Resource Space: 100%|██████████| 3/3 [00:00<00:00, 7635.26it/s]
1.3.5  Multi-dimensional ACM Computing Classification System: 100%|██████████| 3/3 [00:00<00:00, 2034.10it/s]
1.3.7 Media Content Space: 100%|██████████| 1/1 [00:00<00:00, 2187.95it/s]
1.3.8 Automatically Add New Resources to the Resource Space: 100%|██████████| 17/17 [00:00<00:00, 7830.35it/s]
1.4.3 From Low Dimension to High Dimension: 100%|██████████| 17/17 [00:00<00:00, 7617.05it/s]
1.4.4 Abstraction and Analogy in Designing Resource Space: 100%|██████████| 20/20 [00:00<00:00, 7757.17it/s]
1.5 Use Resource Space to Manage Relational Tables: 100%|██████████| 8/8 [00:00<00:00, 7745.71it/s]
1.9 Summary: 100%|██████████| 13/13 [00:00<00:00, 7387.34it/s]
2 A Semantic Overlay Integrating Normalization with Autonomy: 100%|

In [254]:
# 对句子进行信息量排序
# 方案1: 直接用重要度排序
# 方案2: PageRank类方法

# 所有包含基础概念的句子，进行排序
# TODO: 移除所有已经选过的句子，并降权
# TODO: 输出加粗关键概念


def get_importance_of_a_sentence(sentence, words, query_concepts):
    # TODO: 用importances字典
    if len(words) == 0:
        return 0
    score = sum(get_idf_value(concept) for word in words if word not in query_concepts)
    score += sum(importances[concept] for word in words if word in query_concepts)
    
    return score / len(words)
    
global_selected_sentence = set()

def summarize(links, concepts: set, sentence_count = 1):
    # 过滤出所有包含指定concepts的句子子集
    # TODO: concepts变为依赖关系，目前没有考虑【依赖关系】
    filtered_sentences = []
    selected_position = set()
    for link in links:
        if link.pre not in concepts and link.post not in concepts:
            continue
        if link.position in selected_position:
            continue
        selected_position.add(link.position)
        sentence, words = position_section_mapper[link.position]
        
        # TODO: 句子不会重复选择，更好的策略
        if sentence in global_selected_sentence:
            continue
        # global_selected_sentence.add(sentence)
        
        # TODO: sentence的评价标准没有加入和query concepts之间的关联，或者说get_importance_of_a_sentence的关联度不强
        filtered_sentences.append((sentence, words, link))

    # 按照position对所有句子进行排序 (sentence, words, link)
    ranked_sentences = sorted(filtered_sentences, key=lambda item: -get_importance_of_a_sentence(item[0], item[1], concepts))
    
    result_sentences = []
    for sentence, words, link in ranked_sentences[:sentence_count]:
        result_sentences.append(sentence)
        global_selected_sentence.add(sentence)
    return result_sentences

sentences = []

# 算上roots以外，不在core concepts中的概念（sequence的结尾也是core concepts）
basic_concepts = (set(common_history_forest.string_to_node.keys()) - set(core_concepts)) | set([root.content for root in common_history_forest.roots])

# linking concepts是basic concepts和最核心core concepts之前存在依赖关系的concepts
# TODO: 排序core concepts
linking_concepts = set(common_history_forest.string_to_node.keys()) - set(basic_concepts)
# 有多少条item，就有多少个链接路
top_5_concepts = sorted(core_concepts, key=lambda c: -importances.get(c, 0))[:5]
for query_concept in top_5_concepts:
    linking_sequences = []
    for from_concept in linking_concepts:
        linking_sequence = core_dependency_forest.get_node_sequence(from_concept, query_concept, forward=False)
        linking_sequences.append(linking_sequence)

# rest_core_concepts是最最核心的概念，及之后的概念。
# 暂时不考虑分点，直接用top5作为分点
# rest_core_concepts = core_concepts - linking_concepts

content_sequences = []
for query_concept in top_5_concepts:
    sequence = core_dependency_forest.get_node_sequence(query_concept, '', forward=True)
    content_sequences.append(sequence)

# 生成摘要
summary = []
background_sentences = summarize(main_links, basic_concepts, 2)
summary += background_sentences
summary.append('='*30)

for index, (query_concept, linking_sequence, content_sequence) in enumerate(zip(top_5_concepts, linking_sequences, content_sequences), 1):
    summary.append(f'{index}. {query_concept}')
    # sequence转为concepts，逐个学习路径
    linking_concepts = set()
    for sequence in linking_sequence:
        linking_concepts.update(node.content for node in sequence)
    
    motivation_sentences = summarize(main_links, linking_concepts, 2)
    summary += motivation_sentences
    summary.append('-'*30)
    
    # sequence转为concepts
    for sequence in content_sequence:
        content_concepts = set(node.content for node in sequence)
    content_sentences = summarize(main_links, content_concepts, 2)
    summary += content_sentences
    summary.append('='*30)


A second-normal-form resource space is a first-normal-form, and for any axis, any two coordinates are independent from each other.
Chapter 9 constructs the probabilistic Resource Space Model to deal with uncertainty in applications by introducing the probability into the Resource Space Model.
1. resource space
According to definition 3.1, an operation set OP on resource spac-es is complete if: for any given finite collection of resource spaces RS1, RS2, …, RSN in simple normal form, and an operation set OP on resource spac-es can get all the resource spaces in the form of: {RS(x1,  ,xd)| xk RSi (Xj), 1i N, d1 and 1k d }.
So for any given fi-nite collection of resource spaces RS1, RS2, …, RSN in simple normal form, we can get that all the possible query results are in the form of {RS(x1,  , xd)| xk  RSi(Xj), 1iN, d1 and 1kd}, i.e., all the combi-nations of the coordinates of the resource spaces.
------------------------------
A resource space RS(X1, …, Xm, Y1, …, Yn, Z1, …

In [265]:
import termcolor

# TODO: 也可以作为评估指标，出现陌生概念的比例
# 绿色：已经学会的概念，在history概念集中
green_text = lambda s: termcolor.colored(s, 'green')
# 蓝色：在【已学会-核心】概念之间的概念，common
blue_text = lambda s: termcolor.colored(s, 'blue')
# 红色：完全未知的核心概念
red_text = lambda s: termcolor.colored(s, 'red')

from termcolor import colored

def colored_sentence(words: list[str]):
    result = []
    
    for word in words:
        if word in core_concepts:
            word = red_text(word)
            
        if word in linking_concepts:
            word += blue_text(word)

        if word in basic_concepts:
            word += green_text(word)
            
        result.append(word)
        
    return ' '.join(result)

for sentence in summary:
    print(colored_sentence(nltk.tokenize.word_tokenize(sentence)))

A second-normal-form resource space is a first-normal-form , and for any axis , any two coordinates are independent from each other .
Chapter 9 constructs the probabilistic Resource Space Model to deal with uncertainty in applications by introducing the probability into the Resource Space Model .
1. resource space
According to definition 3.1 , an operation set OP on resource spac-es is complete if : for any given finite collection of resource spaces RS1 , RS2 , … , RSN in simple normal formform , and an operation set OP on resource spac-es can get all the resource spaces in the formform of : { RS ( x1 ,  , xd ) | xk RSi ( Xj ) , 1i N , d1 and 1k d } .
So for any given fi-nite collection of resource spaces RS1 , RS2 , … , RSN in simple normal formform , we can get that all the possible query results are in the formform of { RS ( x1 ,  , xd ) | xk  RSi ( Xj ) , 1iN , d1 and 1kd } , i.e. , all the combi-nations of the coordinates of the resource spaces .
-- -- -- -- -- -- --

In [263]:
linking_concepts

('Files are expanding in our daily-use PCs or laptops due to easier down-load from websites and email attachments.',
 ['Files',
  'are',
  'expanding',
  'in',
  'our',
  'daily-use',
  'PCs',
  'or',
  'laptops',
  'due',
  'to',
  'easier',
  'down-load',
  'from',
  'websites',
  'and',
  'email',
  'attachments',
  '.'])

In [142]:
common_history_forest.string_to_node

{'databases': <__main__.DependencyNode at 0x1506bb3d0>,
 'resolution': <__main__.DependencyNode at 0x150475390>,
 'head': <__main__.DependencyNode at 0x150477450>,
 'oid': <__main__.DependencyNode at 0x1500eb890>,
 'form': <__main__.DependencyNode at 0x1500e9250>,
 'semantic information': <__main__.DependencyNode at 0x150324650>,
 'designer': <__main__.DependencyNode at 0x150327850>,
 'exercise': <__main__.DependencyNode at 0x150324f90>,
 'detail': <__main__.DependencyNode at 0x150327e50>,
 'theorem': <__main__.DependencyNode at 0x1503268d0>,
 'chapter': <__main__.DependencyNode at 0x150474b50>,
 'ptime': <__main__.DependencyNode at 0x150325ed0>,
 'space': <__main__.DependencyNode at 0x1503257d0>,
 'part': <__main__.DependencyNode at 0x150324b50>,
 'way': <__main__.DependencyNode at 0x150327810>,
 'cell': <__main__.DependencyNode at 0x150324e90>,
 'neighbors': <__main__.DependencyNode at 0x150325610>,
 'example': <__main__.DependencyNode at 0x150325c50>,
 'body': <__main__.DependencyNo

评估课程大纲

1. 用大模型对比，是否符合那几个准则
2. ROUGE评分：用一组论文集合的abstract全部内容


没有现成的数据集，我们首先用多文本摘要数据集进行评估，科技文档体裁，单词长度分类

短文：合成数据集
长文：多文档

1、ROUGE

多文档摘要

    1） 合成数据集：Co
    2） BigSurvey
    3） 论文


2、13B大模型评估多个维度指标
    * 连贯性
    * 

3、目标应用场景：
论文集合，按照时间进行排序，前面的多少篇论文
交叉混合相同位置的句子

### 对比模型

* 抽取式摘要
    - kl
    - textrank

* 小模型
Phi-1.5B
Llama3-7B
Llama3-13B


In [257]:
from termcolor import colored

print(colored('红色', 'red'))

红色
